In [14]:
import tensorflow as tf
import numpy as np
import random
import sys
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import Callback
from tqdm import tqdm
import time
import sys

# establecer semillas
seed = 4734
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
# Verificar que TensorFlow detecta las GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Configurar TensorFlow para permitir el crecimiento de la memoria GPU
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
else:
    print("GPUs Not Available")


if gpus:
    print(f'Número de GPUs disponibles: {len(gpus)}')
    for i, gpu in enumerate(gpus):
        print(f'Detalles de GPU {i}: {gpu}')
else:
    print('No se detectaron GPUs, se utilizará la CPU.')

Número de GPUs disponibles: 4
Detalles de GPU 0: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Detalles de GPU 1: PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')
Detalles de GPU 2: PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')
Detalles de GPU 3: PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')


2024-07-03 07:37:26.919276: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-07-03 07:37:26.921101: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-07-03 07:37:26.961754: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: Tesla P100-SXM2-16GB computeCapability: 6.0
coreClock: 1.4805GHz coreCount: 56 deviceMemorySize: 15.89GiB deviceMemoryBandwidth: 681.88GiB/s
2024-07-03 07:37:26.962104: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:06:00.0 name: Tesla P100-SXM2-16GB computeCapability: 6.0
coreClock: 1.4805GHz coreCount: 56 deviceMemorySize: 15.89GiB deviceMemoryBandwidth: 681.88GiB/s
2024-07-03 07:37:26.962438: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:84:00.0 name: 

### DATOS

In [5]:
# Cargar datos numpy normalizados - imagenes y velocidades

# Rutas de trabajo
npy_folder = '/disk2/alma/tesisSpace/'
numpy_file_name_image = "numpyFile_simulation_images"
numpy_file_name_velocity_x = "./numpyFile_simulation_velocity_x"
numpy_file_name_velocity_y = "./numpyFile_simulation_velocity_y"

def load_normalize_data():
    
    # Mensaje de depuración
    print("Ready to load NUMPY images_normalize from " + npy_folder)
    sys.stdout.flush()

    # Carga del archivo de imagenes
    try:
        data_norm_img = np.load(npy_folder + numpy_file_name_image + ".npy", mmap_mode=None)
    except:
        print("ERROR loading file: " + npy_folder + numpy_file_name_image + ".npy")
        sys.stdout.flush()
        sys.exit(1)
    else:
        print("The file " + npy_folder + numpy_file_name_image + ".npy was loaded.")
        sys.stdout.flush()

    # Carga del archivo de velocidades por el momento se toman velocidades verticales y horizontales por separado (Velocidades del gas)

    try:
        data_norm_vx = np.load(npy_folder + numpy_file_name_velocity_x + ".npy", mmap_mode=None)
    except:
        print("ERROR loading file: " + npy_folder + numpy_file_name_velocity_x + ".npy")
        sys.stdout.flush()
        sys.exit(1)
    else:
        print("The file " + npy_folder + numpy_file_name_velocity_x + ".npy was loaded.")
        sys.stdout.flush()

    try:
        data_norm_vy = np.load(npy_folder + numpy_file_name_velocity_y + ".npy", mmap_mode=None)
    except:
        print("ERROR loading file: " + npy_folder + numpy_file_name_velocity_y + ".npy")
        sys.stdout.flush()
        sys.exit(1)
    else:
        print("The file " + npy_folder + numpy_file_name_velocity_y + ".npy was loaded.")
        sys.stdout.flush()

    
    # Retorno de las matrices numpy
    return data_norm_img, data_norm_vx, data_norm_vy

In [10]:

def adapt_training_data_Keras(data_img_norm, data_vel_norm):
    print("\n ADAPTING DATA FOR KERAS___________________________")
    
    # Imprimir detalles iniciales del array de datos
    print("I1 dtype =", data_img_norm.dtype, "- shape =", data_img_norm.shape)
    print("V1 dtype =", data_vel_norm.dtype, "- shape =", data_vel_norm.shape)
    
    # Expandir el array de imágenes para incluir el canal como una dimensión adicional
    # (batch_size, time_step, pix, pix) ---> (batch_size, time_step, pix, pix, 1)
    data_img_norm = np.expand_dims(data_img_norm, axis=-1)
    print("I2 dtype =", data_img_norm.dtype, "- shape =", data_img_norm.shape)

    # Expandir el array de velocidades para incluir el canal como una dimensión adicional
    # (batch_size, pix, pix) ---> (batch_size, pix, pix, 1)
    data_vel_norm = np.expand_dims(data_vel_norm, axis=-1)
    print("V2 dtype =", data_vel_norm.dtype, "- shape =", data_vel_norm.shape)

    print("\n")
    
    # Inicialización de listas de índices
    random_list_indexes_train = []
    random_list_indexes_valid = []
    random_list_indexes_test = []
    
    total_data_len = data_img_norm.shape[0]

    # Crear índices para dividir los datos
    indices = np.arange(total_data_len)
    
    # Dividir los datos en conjuntos de entrenamiento + validación y prueba
    random_indexes_valid, random_indexes_test = train_test_split(
        indices, test_size=0.2, random_state=23)
    
    # Dividir los datos de entrenamiento + validación en entrenamiento y validación
    random_indexes_train, random_indexes_valid = train_test_split(
        random_indexes_valid, test_size=0.25, random_state=23)
    
    # Selección de imágenes para los conjuntos de entrenamiento, validación y prueba
    train_images = data_img_norm[random_indexes_train]
    valid_images = data_img_norm[random_indexes_valid]
    test_images = data_img_norm[random_indexes_test]
    print("Train_images dtype =", train_images.dtype, "- shape =", train_images.shape)
    print("Valid_images =", valid_images.dtype, "- shape =", valid_images.shape)
    print("Test_images =", test_images.dtype, "- shape =", test_images.shape)
    print("\n")

    train_velocitys = data_vel_norm[random_indexes_train]
    valid_velocitys = data_vel_norm[random_indexes_valid]
    test_velocitys = data_vel_norm[random_indexes_test]
    print("Train_velocity dtype =", train_velocitys.dtype, "- shape =", train_velocitys.shape)
    print("Valid_velocity dtype =", valid_velocitys.dtype, "- shape =", valid_velocitys.shape)
    print("Test_velocity dtype =", test_velocitys.dtype, "- shape =", test_velocitys.shape)
    
    # No hay etiquetas, así que solo se devuelven las imágenes y velocidades
    return train_images, valid_images, test_images, train_velocitys, valid_velocitys, test_velocitys, total_data_len


In [7]:
# Preparar dataset para el entrenamiento

def prepare_datasets(train_images, valid_images, test_images, train_velocity, valid_velocity, test_velocity, batch_size):
    print("\n READY TO PREPARE DATASETS______________________")
    
    # Convertir a tensores de TensorFlow
    train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_velocity)).batch(batch_size).shuffle(buffer_size=len(train_images))
    valid_dataset = tf.data.Dataset.from_tensor_slices((valid_images, valid_velocity)).batch(batch_size)
    test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_velocity)).batch(batch_size)
    
    print("Train_dataset = ", train_dataset)
    print("Valid_dataset = ", valid_dataset)
    print("Test_dataset = ", test_dataset)
    print("\n")
    
    return train_dataset, valid_dataset, test_dataset


In [27]:
# Procesar datos

#Cargar datos (batch_size, time_step, pix, pix) image
data_img_norm, data_vx_norm, data_vy_norm = load_normalize_data()

pix_prueba = 60

time_steps = data_img_norm.shape[1]
height = pix_prueba
width = pix_prueba
channels = 1

#Prueba simple
data_img_norm = data_img_norm[:, :, 0:pix_prueba, 0:pix_prueba]
data_vy_norm = data_vy_norm[:, 0:pix_prueba, 0:pix_prueba]

print("Image Shape", data_img_norm.shape)
print("Image Max: ", data_img_norm.max(), " Image Min: ", data_img_norm.min())

print("Vx Shape", data_vx_norm.shape)
print("Vx Max: ", data_vx_norm.max(), " Vx Min: ", data_vx_norm.min())

print("Vy Shape", data_vy_norm.shape)
print("Vy Max: ", data_vy_norm.max(), " Vy Min: ", data_vy_norm.min())


# Adaptar los datos para PyTorch y generar conjuntos de entrenamiento
train_images, valid_images, test_images, train_velocity, valid_velocity, test_velocity, batch_size = \
adapt_training_data_Keras(data_img_norm, data_vy_norm)


# Crear datasets
train_dataset, valid_dataset, test_dataset = prepare_datasets(train_images, valid_images, test_images,
                                                          train_velocity, valid_velocity, test_velocity,
                                                          batch_size=batch_size)

Ready to load NUMPY images_normalize from /disk2/alma/tesisSpace/
The file /disk2/alma/tesisSpace/numpyFile_simulation_images.npy was loaded.
The file /disk2/alma/tesisSpace/./numpyFile_simulation_velocity_x.npy was loaded.
The file /disk2/alma/tesisSpace/./numpyFile_simulation_velocity_y.npy was loaded.
Image Shape (77, 19, 60, 60)
Image Max:  0.5261954  Image Min:  0.0
Vx Shape (77, 300, 300)
Vx Max:  1.0  Vx Min:  0.0
Vy Shape (77, 60, 60)
Vy Max:  1.0  Vy Min:  0.0

 ADAPTING DATA FOR KERAS___________________________
I1 dtype = float32 - shape = (77, 19, 60, 60)
V1 dtype = float32 - shape = (77, 60, 60)
I2 dtype = float32 - shape = (77, 19, 60, 60, 1)
V2 dtype = float32 - shape = (77, 60, 60, 1)


Train_images dtype = float32 - shape = (45, 19, 60, 60, 1)
Valid_images = float32 - shape = (16, 19, 60, 60, 1)
Test_images = float32 - shape = (16, 19, 60, 60, 1)


Train_velocity dtype = float32 - shape = (45, 60, 60, 1)
Valid_velocity dtype = float32 - shape = (16, 60, 60, 1)
Test_velo

### MODELO

In [32]:
import tensorflow as tf

# Definir una estrategia de distribución
strategy = tf.distribute.MirroredStrategy(devices=['/gpu:0', '/gpu:1', '/gpu:2', '/gpu:3'])

class DistributedDeepSanneModel(tf.keras.Model):
    def __init__(self, time_steps, height, width, channels):
        super(DistributedDeepSanneModel, self).__init__()
        self.time_steps = time_steps
        self.height = height
        self.width = width
        self.channels = channels
        
        # Modelo secuencial para las capas asignadas a las GPUs
        self.model = tf.keras.Sequential([
            # Capa ConvLSTM2D para asignar a la GPU 0
            tf.keras.layers.ConvLSTM2D(
                filters=64,
                kernel_size=(3, 3),
                strides=(1, 1),
                padding='same',
                activation='relu',
                input_shape=(self.time_steps, self.height, self.width, self.channels)
            ),
            
            # Capa Conv2D para asignar a la GPU 1
            tf.keras.layers.Conv2D(
                filters=128,
                kernel_size=(3, 3),
                strides=(1, 1),
                padding='same',
                activation='relu'
            ),
            
            # Capa Flatten para asignar a la GPU 2
            tf.keras.layers.Flatten(),
            
            # Capa Dense para asignar a la GPU 3
            tf.keras.layers.Dense(128, activation='relu'),
            
            # Capa de salida para asignar a la GPU 0
            tf.keras.layers.Dense(1, activation='linear')
        ])
    
    def call(self, inputs):
        return self.model(inputs)

# Instanciar modelo dentro de la estrategia de distribución
with strategy.scope():
    model = DistributedDeepSanneModel(time_steps=time_steps, height=height, width=width, channels=channels)

# Compilar modelo 
model.compile(optimizer='adam', loss='mse')

# Mostrar la arquitectura del modelo
model.model.summary()


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_6 (ConvLSTM2D)  (None, 60, 60, 64)        150016    
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 60, 60, 128)       73856     
_________________________________________________________________
flatten_6 (Flatten)          (None, 460800)            0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               58982528  
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 129       
Total p

In [41]:
### Distribucion manual y controlada de tensores en dispositivos.

# Definir una estrategia de distribución
strategy = tf.distribute.MirroredStrategy(devices=['/gpu:0', '/gpu:1', '/gpu:2', '/gpu:3'])

class DistributedDeepSanneModel(tf.keras.Model):
    def __init__(self):
        super(DistributedDeepSanneModel, self).__init__()
        
        # Asignar la primera capa a la GPU 0
        with tf.device('/gpu:0'):
            self.dense1 = tf.keras.layers.Dense(64, activation='linear', input_shape=(time_steps, height, width, channels))
        
        # Asignar la segunda capa a la GPU 1
        with tf.device('/gpu:1'):
                self.convLSTM = tf.keras.layers.ConvLSTM2D(
                filters=128,  # Canales de salida 128
                kernel_size=(4, 4),  # tamaño del kernel 4x4
                strides=(1, 1),  # por defecto
                padding='same',  # mantener dimensionalidad entrada salida
                activation='relu',  # función de activacion ReLU
                return_sequences=False,  # retorna solo la última salida en la secuencia de salidas
                recurrent_activation='hard_sigmoid' 
            )
        
        # Asignar la tercera capa a la GPU 2
        with tf.device('/gpu:2'):
            self.dense2 = tf.keras.layers.Dense(128, activation='linear')
        
        # Asignar la cuarta capa a la GPU 3
        with tf.device('/gpu:3'):
            self.dense3 = tf.keras.layers.Dense(1, activation='linear')
    
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.convLSTM(x)
        x = self.dense2(x)
        return self.dense3(x)

# Crear una instancia del modelo dentro del alcance de la estrategia
with strategy.scope():
    modelTF = DistributedDeepSanneModel()
    modelTF.build((None, time_steps, height, width, channels))  # Construir el modelo con una entrada dummy
    
    # Compilar el modelo (se puede personalizar según tus necesidades)
    modelTF.compile(optimizer='adam', loss='mse')

# Mostrar la arquitectura del modelo
modelTF.summary()


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Model: "distributed_deep_sanne_model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             multiple                  128       
_________________________________________________________________
conv_lst_m2d_10 (ConvLSTM2D) multiple                  1573376   
_________________________________________________________________
dense_32 (Dense)             multiple                  16512     
_________________________________________________________________
dense_33 (Dense)             multiple                  129       
Total params: 1,590,145
Trainable params: 1,590,145
Non-trainable params: 0
____________________________________________

### ENTRENAMIENTO

In [36]:
# Dentro del ciclo de entrenamiento, después de cada batch
for inputs, targets in train_dataset:
    # Hacer una predicción con el modelo
    predictions = model(inputs)
    
    # Imprimir las formas de las salidas de las capas intermedias
    for layer in model.layers:
        intermediate_output = layer(inputs)
        print(f"Layer {layer.name} output shape: {intermediate_output.shape}")
    
    # Continuar con el entrenamiento normalmente


Layer sequential_9 output shape: (45, 1)


In [35]:

# Clase de callback para mostrar progreso con tqdm
class TQDMCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        self.epoch += 1
        self.tqdm.update(1)
        self.tqdm.set_postfix(logs)

    def on_train_begin(self, logs=None):
        self.epoch = 0
        self.tqdm = tqdm(total=self.params['epochs'], desc="Training", unit="epoch")

    def on_train_end(self, logs=None):
        self.tqdm.close()


# Definir una estrategia de distribución
strategy = tf.distribute.MirroredStrategy(devices=['/gpu:0', '/gpu:1', '/gpu:2', '/gpu:3'])

# Crear y compilar el modelo dentro de la estrategia
with strategy.scope():
    model = DistributedDeepSanneModel(time_steps=time_steps, height=height, width=width, channels=channels)
    model.compile(optimizer=Adam(lr=0.001), loss=MeanSquaredError())

    # Mostrar el resumen del modelo
    model.model.summary()

    # Definir callbacks, como el de progreso con tqdm
    callbacks = [TQDMCallback()]

    # Entrenar el modelo
    epochs = 10
    history = model.fit(train_dataset, epochs=epochs, validation_data=valid_dataset, callbacks=callbacks)

    # Evaluar en el conjunto de prueba
    test_loss = model.evaluate(test_dataset)
    print(f"Test Loss: {test_loss:.4f}")

    # Guardar los resúmenes de entrenamiento, validación y prueba
    train_summaries = history.history['loss']
    valid_summaries = history.history['val_loss']

    with open('train_summaries.txt', 'w') as f:
        for epoch, loss in enumerate(train_summaries, 1):
            f.write(f"Epoch {epoch}, Training Loss: {loss:.4f}\n")

    with open('valid_summaries.txt', 'w') as f:
        for epoch, loss in enumerate(valid_summaries, 1):
            f.write(f"Epoch {epoch}, Validation Loss: {loss:.4f}\n")

    with open('test_summaries.txt', 'w') as f:
        f.write(f"Test Loss: {test_loss:.4f}\n")




INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


2024-07-03 08:06:35.293974: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 19
        }
        dim {
          size: 60
        }
        dim {
          size: 60
        }
        dim {
          size: 1
        }
      }
      shape {
        dim {
          size: 60
        }
        dim {
          size: 60
        }
        dim {
          size: 1
        }
      }
    }
  }
}



Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_9 (ConvLSTM2D)  (None, 60, 60, 64)        150016    
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 60, 60, 128)       73856     
_________________________________________________________________
flatten_9 (Flatten)          (None, 460800)            0         
_________________________________________________________________
dense_18 (Dense)             (None, 128)               58982528  
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 129       
Total params: 59,206,529
Trainable params: 59,206,529
Non-trainable params: 0
_________________________________________________________________




Training:   0%|                                                                              | 0/10 [00:00<?, ?epoch/s]

Epoch 1/10


Training:   0%|                                                                              | 0/10 [01:40<?, ?epoch/s]


INFO:tensorflow:batch_all_reduce: 9 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 9 all-reduces with algorithm = nccl, num_packs = 1


InvalidArgumentError: 5 root error(s) found.
  (0) Invalid argument:  Incompatible shapes: [9,60,60,1] vs. [9,1]
	 [[node replica_3/mean_squared_error/SquaredDifference (defined at disk2/alma/anaconda3/envs/tf-gpu/lib/python3.9/threading.py:980) ]]
	 [[div_no_nan/ReadVariableOp_3/_680]]
  (1) Invalid argument:  Incompatible shapes: [9,60,60,1] vs. [9,1]
	 [[node replica_3/mean_squared_error/SquaredDifference (defined at disk2/alma/anaconda3/envs/tf-gpu/lib/python3.9/threading.py:980) ]]
  (2) Invalid argument:  Incompatible shapes: [9,60,60,1] vs. [9,1]
	 [[node replica_3/mean_squared_error/SquaredDifference (defined at disk2/alma/anaconda3/envs/tf-gpu/lib/python3.9/threading.py:980) ]]
	 [[div_no_nan/_699]]
  (3) Invalid argument:  Incompatible shapes: [9,60,60,1] vs. [9,1]
	 [[node replica_3/mean_squared_error/SquaredDifference (defined at disk2/alma/anaconda3/envs/tf-gpu/lib/python3.9/threading.py:980) ]]
	 [[Adam/Adam/group_deps/NoOp_1/_747]]
  (4) Invalid argument:  Incompatible s